In [1]:
import pickle

#### О данных, на которых делали пробинг грамматических категорий в BERT:

1. en_ewt -- Genre: blog, social, reviews, email, web

2. en_gum -- Genre: academic, blog, fiction, government, news, nonfiction, social, spoken, web, wiki

#### Какие категории были:

* Общие для двух датасетов

1. Case
* Nom (The point was; He said; I said)
* Gen (Despite Daniel' s attempts; Blood smeared the back of his hand)
* Acc (Wash the leaves; Got it)

    То есть тут существительные и местоимения.

2. Gender
* Masc (Dvořák ' s first piece; He / his)
* Fem (Kiara Perkins; She joined)
* Neut (In other words; It is unsurprising)

    Тут существительные собственные и нарицательные, местоимения.

3. Definite
* Ind (When regarding an artwork; a)
* Def (I shall leave this office; the)

    Тут артикли и указательные местоимения.

4. Number
* Sing (As a foreign visitor; Gonna rip your purse; Seen this before; I know it)
* Plur (Six months; See what else we got here; Let' s just double check that)

    Тут существительные, местоимения, глаголы в каких то случаях кажется. 

5. Person
* 1 (I / We)
* 2 (You / Your; Just be aware)
* 3 (всякое разное)

    Тут местоимения, глагол.

6. Degree
* Pos (Although useful; Some disciplines; In computer science)
* Cmp (much smaller; more details; sooner or later; explain more)
* Sup (On the hottest days; Most of the rooms)

    Тут прилагательные, неопределенное местоимение, наречия (?). 

7. NumForm

* Digit (13; + всякая каша; Ø -- символы)
* Word (one; five; + каша)
* Roman (I, II)

    Числительные.

8. NumType
* Ord (first; 17th; + каша)
* Card (1; 2014; one)
* Mult (once; twice) -- мало их
* Frac (2.4) -- мало их

    Числительные.

9. Mood
* Ind
* Imp (Press okay)
* Sub (только в EWT и очень мало)

    Наклонение.

10. Tense
* Past
* Pres

    Кажется тут только время глагола.

11. VerbForm
* Fin 
* Part 
* Inf 
* Ger 



* Различающиеся категории датасетов

12. PronType (GUM)
* Neg 
* Dem 
* Tot 
* Rel 
* Int 
* Prs 
* Emp 
* Ind 
* Art 

    Для EWT и GUM было предупреждение The classes in train and validation parts are different for category "PronType", но с EWT и в итоге падала ошибка на пробинге.
    
12. ExtPos (EWT)
* ADV 
* CCONJ 
* PRON 
* SCONJ 
* ADP (figures such as the Pope (such = ADJ))

#### Как делали: 

При помощи NeuroX tooklit

1. Даем conllu файлы AIRI Probing_framework
2. Конвертируем .csv файлы в .txt формат и делаем разные файлы для train и test. 
* ограничиваю длину предложения: 5 < len(sentence.split()) < 40
* беру максимум 2600 предложений для train и 900 для test (эмбеддинги очень много весят) в каждой категории.
* делаю balanced-sampling, чтобы отбирать одинаковое количество предложений для каждого лейбла в грамматической категории

! Ошибка -- минимальный порог слишком высок. Надо ставить длину 2-3, а не 5.

! Проблема -- недостаточное количество данных для некоторых меток. Возможно, отчасти из-за ограничений на длину предложения, но также потому что их просто недостаточно. 
Гипотетическое решение: подумать, как вытаскивать эти данные из неиспользуемых train-данных.
* NeuroX дает эмбеддинги для каждого предложения в train и test.
3. Подаю данные в NeuroX: 
* linear_probe.train_logistic_regression_probe -- тренируем
* linear_probe.evaluate_probe -- получаю accuracy для train и test. 
Гипотетически можно поставить F1, среднее accuracy и F1, pearson.
* получаю ранжирование по нейронам.

### Загружаем метрики и ранжирование нейронов

In [2]:
with open('scores_ewt.pkl', 'rb') as f:
    scores_ewt = pickle.load(f)
with open('neurons_ewt.pkl', 'rb') as f:
    ordered_neurons_ewt = pickle.load(f)
with open('scores_gum.pkl', 'rb') as f:
    scores_gum = pickle.load(f)
with open('neurons_gum.pkl', 'rb') as f:
    ordered_neurons_gum = pickle.load(f)

In [3]:
def bad_scores(scores):
    for k, v in scores.items():
        m1 = []
        m2 = []

        for i, j in v[0].items():

            if v[0][i] < 0.5:

                if not m1.__contains__(v[0]):
                    m1.append(v[0])

                print(k, i, 'train_score', v[0][i])
        if m1:
            print(k, 'train', m1)
            print('---------------------')

        for i, j in v[1].items():
            if v[1][i] < 0.5:

                if not m2.__contains__(v[1]):
                    m2.append(v[1])

                print(k, i, 'test_score', v[1][i])
        if m2:
            print(k, 'test', m2)
            print('---------------------')

### Плохая классификация для датасета en_ewt (accuracy < 0.5)

Genre: blog, social, reviews, email, web

In [4]:
bad_scores(scores_ewt)

VerbForm __OVERALL__ test_score 0.2490372272143774
VerbForm Fin test_score 0.49333333333333335
VerbForm Part test_score 0.1288888888888889
VerbForm Inf test_score 0.008888888888888889
VerbForm test [{'__OVERALL__': 0.2490372272143774, 'Fin': 0.49333333333333335, 'Ger': 0.5, 'Part': 0.1288888888888889, 'Inf': 0.008888888888888889}]
---------------------
NumForm Roman test_score 0.0
NumForm test [{'__OVERALL__': 0.7839080459770115, 'Word': 0.7251908396946565, 'Roman': 0.0, 'Digit': 0.82}]
---------------------
ExtPos CCONJ test_score 0.25
ExtPos PRON test_score 0.0
ExtPos SCONJ test_score 0.0
ExtPos ADP test_score 0.36363636363636365
ExtPos test [{'__OVERALL__': 0.5945945945945946, 'ADV': 0.85, 'CCONJ': 0.25, 'PRON': 0.0, 'SCONJ': 0.0, 'ADP': 0.36363636363636365}]
---------------------
Mood Sub train_score 0.2
Mood train [{'__OVERALL__': 0.9910884786760026, 'Ind': 0.9965397923875432, 'Sub': 0.2, 'Imp': 0.9899856938483548}]
---------------------
Mood Sub test_score 0.0
Mood test [{'__OVER

### Вывод после пробинга на en_ewt:

Для некоторых лейблов в грамматических категориях ОЧЕНЬ плохая метрика test_score. Потому что там было очень мало данных. 

Крайний пример:

Mood Sub -- плохая метрика и в train (0.2), и в test (0.0) -- 5 предложений в train и 2 в test. 

OVERALL плохая метрика -- для VerbForm (test_score 0.249). 
Хотя там было сравнительно нормальное количество данных -- 2188 в train и 779 в test. Но в категории 4 лейбла -- значит надо больше данных. 

### Плохая классификация для датасета en_gum

Genre: academic, blog, fiction, government, news, nonfiction, social, spoken, web, wiki

In [5]:
bad_scores(scores_gum)

VerbForm Fin test_score 0.4088888888888889
VerbForm test [{'__OVERALL__': 0.6628571428571428, 'Inf': 0.874439461883408, 'Part': 0.7155555555555555, 'Ger': 0.5925925925925926, 'Fin': 0.4088888888888889}]
---------------------
NumForm Roman test_score 0.0
NumForm test [{'__OVERALL__': 0.6887052341597796, 'Word': 0.7959183673469388, 'Digit': 0.6273584905660378, 'Roman': 0.0}]
---------------------
PronType Neg test_score 0.0
PronType Rel test_score 0.14285714285714285
PronType Rcp test_score 0.0
PronType test [{'__OVERALL__': 0.682648401826484, 'Neg': 0.0, 'Dem': 0.7777777777777778, 'Tot': 0.5833333333333334, 'Rel': 0.14285714285714285, 'Int': 0.8688524590163934, 'Prs': 0.7333333333333333, 'Rcp': 0.0, 'Ind': 0.5714285714285714, 'Art': 0.7222222222222222}]
---------------------
Gender Fem test_score 0.015873015873015872
Gender Masc test_score 0.1592356687898089
Gender test [{'__OVERALL__': 0.5123595505617977, 'Fem': 0.015873015873015872, 'Masc': 0.1592356687898089, 'Neut': 0.89777777777777

### Вывод после пробинга на en_gum:


Нет Overall плохих метрик для целой категории.

Есть плохие метрики для лейблов в категориях.

Как и в en_ewt: плохие метрики для лейблов в Gender (Fem, Masc), NumType (Mult, Frac), NumForm (Roman??).

Внезапное, что не было плохо на en_ewt: Degree (Cmp, Sup), Case (Acc).

### Просто посмотреть на N нейронов

В чем суть:
пока простой тупой метод посмотреть общий set(нейронов), если для каждой категории выбираем N-top нейронов в ранжировании. 

Всего нейронов в BERT 9984 (13 слоев * 768 -- размерность эмбеддинга).

Всего в каждом датасете было 12 категорий. 

In [6]:
def get_overall_common_neurons(n, dct):
    number_cats, neurons_list = 0, []
    for k, v in dct.items():
        v = v[:n]
        neurons_list+=v
        number_cats+=1
    f1 = f'All neuron number if {n} for each category -- {len(neurons_list)} among {number_cats} categories'
    f2 = f'Unique neuron number if {n} for each category -- {len(set(neurons_list))} among {number_cats} categories'
    return f1, f2  

#### 10 нейронов

In [7]:
get_overall_common_neurons(10, ordered_neurons_ewt) #EWT

('All neuron number if 10 for each category -- 120 among 12 categories',
 'Unique neuron number if 10 for each category -- 114 among 12 categories')

In [8]:
get_overall_common_neurons(10, ordered_neurons_gum) #GUM

('All neuron number if 10 for each category -- 120 among 12 categories',
 'Unique neuron number if 10 for each category -- 118 among 12 categories')

#### 100 нейронов

In [9]:
get_overall_common_neurons(100, ordered_neurons_ewt) #EWT

('All neuron number if 100 for each category -- 1200 among 12 categories',
 'Unique neuron number if 100 for each category -- 1073 among 12 categories')

In [10]:
get_overall_common_neurons(100, ordered_neurons_gum) #GUM

('All neuron number if 100 for each category -- 1200 among 12 categories',
 'Unique neuron number if 100 for each category -- 1088 among 12 categories')

#### 300 нейронов

In [11]:
get_overall_common_neurons(300, ordered_neurons_ewt) #EWT

('All neuron number if 300 for each category -- 3600 among 12 categories',
 'Unique neuron number if 300 for each category -- 2799 among 12 categories')

In [12]:
get_overall_common_neurons(300, ordered_neurons_gum) #GUM

('All neuron number if 300 for each category -- 3600 among 12 categories',
 'Unique neuron number if 300 for each category -- 2773 among 12 categories')

#### 500 нейронов

In [13]:
get_overall_common_neurons(500, ordered_neurons_ewt) #EWT

('All neuron number if 500 for each category -- 6000 among 12 categories',
 'Unique neuron number if 500 for each category -- 4152 among 12 categories')

In [14]:
get_overall_common_neurons(500, ordered_neurons_gum) #GUM

('All neuron number if 500 for each category -- 6000 among 12 categories',
 'Unique neuron number if 500 for each category -- 4108 among 12 categories')

#### 1000 нейронов

In [15]:
get_overall_common_neurons(1000, ordered_neurons_ewt) #EWT

('All neuron number if 1000 for each category -- 12000 among 12 categories',
 'Unique neuron number if 1000 for each category -- 6476 among 12 categories')

In [16]:
get_overall_common_neurons(1000, ordered_neurons_gum) #GUM

('All neuron number if 1000 for each category -- 12000 among 12 categories',
 'Unique neuron number if 1000 for each category -- 6417 among 12 categories')